# Preparando os dados para o [fine-tuning]

In [ ]:
# Montando Google-Drive:

from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Descompactando LF-Amazon:

#!unzip '/content/drive/MyDrive/Biblioteca/Acadêmico/Pós Graduação/Pós Tech - FIAP/Aulas/Fase_3/Tech_Challenge/Tech_Challenge--5IADT--Fase_03/LF-Amazon-1.3M.raw.zip' -d '/content/drive/MyDrive/Biblioteca/Acadêmico/Pós Graduação/Pós Tech - FIAP/Aulas/Fase_3/Tech_Challenge/Tech_Challenge--5IADT--Fase_03/'

In [ ]:
# Descompactando arquivos json:

#!gunzip '/content/drive/MyDrive/Biblioteca/Acadêmico/Pós Graduação/Pós Tech - FIAP/Aulas/Fase_3/Tech_Challenge/Tech_Challenge--5IADT--Fase_03/LF-Amazon-1.3M/lbl.json.gz'
#!gunzip '/content/drive/MyDrive/Biblioteca/Acadêmico/Pós Graduação/Pós Tech - FIAP/Aulas/Fase_3/Tech_Challenge/Tech_Challenge--5IADT--Fase_03/LF-Amazon-1.3M/trn.json.gz'
#!gunzip '/content/drive/MyDrive/Biblioteca/Acadêmico/Pós Graduação/Pós Tech - FIAP/Aulas/Fase_3/Tech_Challenge/Tech_Challenge--5IADT--Fase_03/LF-Amazon-1.3M/tst.json.gz'

In [ ]:
# Verificando as cinco primeiras linhas do arquivo [trn.json]:

# Defina o caminho para o arquivo de treino que você descompactou
caminho_arquivo_treino = r'/content/drive/MyDrive/Biblioteca/Acadêmico/Pós Graduação/Pós Tech - FIAP/Aulas/Fase_3/Tech_Challenge/Tech_Challenge--5IADT--Fase_03/LF-Amazon-1.3M/trn.json'

# Vamos ler e imprimir apenas as 5 primeiras linhas
print(f"--- Analisando as primeiras 5 linhas de: {caminho_arquivo_treino} ---")
try:
    with open(caminho_arquivo_treino, 'r', encoding='utf-8') as f:
        for i, line in enumerate(f):
            print(f"Linha {i+1}: {line.strip()}")
            if i >= 4:  # Parar depois de 5 linhas (índice 0 a 4)
                break
except Exception as e:
    print(f"Ocorreu um erro ao ler o arquivo: {e}")

In [ ]:
# Verificando as cinco primeiras linhas do arquivo [tst.json]:

# Defina o caminho para o arquivo de treino que você descompactou
caminho_arquivo_treino = r'/content/drive/MyDrive/Biblioteca/Acadêmico/Pós Graduação/Pós Tech - FIAP/Aulas/Fase_3/Tech_Challenge/Tech_Challenge--5IADT--Fase_03/LF-Amazon-1.3M/tst.json'

# Vamos ler e imprimir apenas as 5 primeiras linhas
print(f"--- Analisando as primeiras 5 linhas de: {caminho_arquivo_treino} ---")
try:
    with open(caminho_arquivo_treino, 'r', encoding='utf-8') as f:
        for i, line in enumerate(f):
            print(f"Linha {i+1}: {line.strip()}")
            if i >= 4:  # Parar depois de 5 linhas (índice 0 a 4)
                break
except Exception as e:
    print(f"Ocorreu um erro ao ler o arquivo: {e}")

In [ ]:
# Verificando as cinco primeiras linhas do arquivo [lbl.json]:

# Defina o caminho para o arquivo de treino que você descompactou
caminho_arquivo_treino = r'/content/drive/MyDrive/Biblioteca/Acadêmico/Pós Graduação/Pós Tech - FIAP/Aulas/Fase_3/Tech_Challenge/Tech_Challenge--5IADT--Fase_03/LF-Amazon-1.3M/lbl.json'

# Vamos ler e imprimir apenas as 5 primeiras linhas
print(f"--- Analisando as primeiras 5 linhas de: {caminho_arquivo_treino} ---")
try:
    with open(caminho_arquivo_treino, 'r', encoding='utf-8') as f:
        for i, line in enumerate(f):
            print(f"Linha {i+1}: {line.strip()}")
            if i >= 4:  # Parar depois de 5 linhas (índice 0 a 4)
                break
except Exception as e:
    print(f"Ocorreu um erro ao ler o arquivo: {e}")

# Script de Processamento
> Segundo as instruções do Tech Challenge, serão utilizadas somente duas coluas:
```
...
você utilizará as colunas “title” e
“content”, que contém título e descrição respectivamente.
...
(Tech Challenge.pdf)
```

In [ ]:
import json
import os
import html # Estou usando esta lib para tratar as [Entidades HTML]; quero que o texto seja apresentado corretamente.
import re # Importe a biblioteca de expressões regulares para limpeza extra.

# --- CONFIGURAÇÃO ---
# Caminhos para o Googl-Drive.
DRIVE_BASE_PATH = r"/content/drive/MyDrive/Biblioteca/Acadêmico/Pós Graduação/Pós Tech - FIAP/Aulas/Fase_3/Tech_Challenge/Tech_Challenge--5IADT--Fase_03/LF-Amazon-1.3M"
INPUT_FILE_PATH = os.path.join(DRIVE_BASE_PATH, 'trn.json')
OUTPUT_FILE_PATH = os.path.join(DRIVE_BASE_PATH, 'dataset_para_finetuning.jsonl')

# Defina quantos exemplos você quer processar.
# Comece com um número menor (ex: 20000) para testar o fluxo.
# Para o treino final, você pode aumentar se necessário.
MAX_EXAMPLES = 50000
count = 0

# Template de prompt nativo do Llama-3
LLAMA3_PROMPT_TEMPLATE = """<|begin_of_text|><|start_header_id|>user<|end_header_id|>

Com base no título do produto, gere a sua descrição.
Título: {}<|eot_id|><|start_header_id|>assistant<|end_header_id|>

{}<|eot_id|><|end_of_text|>"""

print("Iniciando o processamento do dataset para o formato nativo Llama-3, com limpeza de HTML...")

# Usamos 'with' para garantir que os arquivos sejam fechados corretamente
with open(INPUT_FILE_PATH, 'r', encoding='utf-8') as infile, \
     open(OUTPUT_FILE_PATH, 'w', encoding='utf-8') as outfile:

    for line in infile:
        # Para o loop quando atingir o número desejado de exemplos
        if count >= MAX_EXAMPLES:
            print(f"Limite de {MAX_EXAMPLES} exemplos atingido. Parando o processamento.")
            break

        try:
            # Carrega a linha atual (que é uma string JSON) para um dicionário Python
            original_record = json.loads(line)

            # Extrai os campos que nos interessam usando .get() para evitar erros
            title = html.unescape(original_record.get('title', ''))
            content = html.unescape(original_record.get('content', ''))

            # --- Passo de Limpeza e Validação Crucial ---
            # Pular registros que não têm título ou que têm conteúdo vazio
            if not title or not content:
                continue

            # --- ETAPA DE LIMPEZA ADICIONAL ---
            # Remove a frase "--This text refers to..." e qualquer variação dela.
            content = re.sub(r'--This text refers to.*', '', content).strip()

            # Se após a limpeza o conteúdo ficar vazio, pule o registro.
            if not content:
                continue
            # --- FIM DA LIMPEZA ---

            # Cria o novo dicionário no formato de instrução que o modelo espera
            # NOTE: A instrução está em português para clareza, enquanto os dados
            # de input/output estão em inglês, aproveitando a capacidade
            # multilingue do modelo.
            # Usamos um único campo "text" que já contém o prompt completo e formatado
            formatted_text = LLAMA3_PROMPT_TEMPLATE.format(title, content)

            # Escrevemos um dicionário contendo apenas a chave "text"
            new_record = {"text": formatted_text}

            outfile.write(json.dumps(new_record, ensure_ascii=False) + '\n')
            count += 1

            # Imprime um status a cada 1000 registros processados
            if count % 5000 == 0:
                print(f"Processados {count} registros...")

        except json.JSONDecodeError:
            # Ignora linhas que não sejam um JSON válido
            continue

print(f"\nProcessamento concluído! {count} exemplos válidos foram salvos em '{OUTPUT_FILE_PATH}'.")

In [ ]:
# Verificando o conteúdo do arquivo gerado:
! head '/content/drive/MyDrive/Biblioteca/Acadêmico/Pós Graduação/Pós Tech - FIAP/Aulas/Fase_3/Tech_Challenge/Tech_Challenge--5IADT--Fase_03/LF-Amazon-1.3M/dataset_para_finetuning.jsonl'

## Conclusão da preparação dos dados
> Foram utilizado 50.000 registros para amostragem.

# Fine-Tuning

In [ ]:
# Montando Google-Drive:

from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Instalando libs necessárias:
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps xformers "trl<0.9.0" peft accelerate bitsandbytes
!pip install transformers datasets

In [ ]:
# Configuração
from unsloth import FastLanguageModel, is_bfloat16_supported

import torch
import json
from datasets import load_dataset
from trl import SFTTrainer
from transformers import TrainingArguments

DATA_PATH = r'/content/drive/MyDrive/Biblioteca/Acadêmico/Pós Graduação/Pós Tech - FIAP/Aulas/Fase_3/Tech_Challenge/Tech_Challenge--5IADT--Fase_03/LF-Amazon-1.3M/dataset_para_finetuning.jsonl'
max_seq_length = 2048
dtype = None
load_in_4bit = True
fourbit_models = [
    "unsloth/mistral-7b-v0.3-bnb-4bit",
    "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    "unsloth/llama-3-8b-bnb-4bit",
    "unsloth/llama-3-8b-Instruct-bnb-4bit",
    "unsloth/llama-3-70b-bnb-4bit",
    "unsloth/Phi-3-mini-4k-instruct",
    "unsloth/Phi-3-medium-4k-instruct",
    "unsloth/mistral-7b-bnb-4bit",
    "unsloth/gemma-7b-bnb-4bit",
]

# Carrega o dataset a partir do arquivo JSON formatado
dataset = load_dataset("json", data_files=DATA_PATH, split="train")

In [ ]:
# Instanciando o Modelo:

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = fourbit_models[2],
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

In [ ]:
# Preparando o Modelo LoRA:

model = FastLanguageModel.get_peft_model(
    model,
    r = 32,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 64,
    lora_dropout = 0,
    bias = "none",

    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
    use_rslora = False,
    loftq_config = None,
)

In [ ]:
# Ajustes finais para treinar o modelo

# O diretório no seu Drive para salvar o modelo final e os checkpoints
output_dir = r"/content/drive/MyDrive/Biblioteca/Acadêmico/Pós Graduação/Pós Tech - FIAP/Aulas/Fase_3/Tech_Challenge/Tech_Challenge--5IADT--Fase_03/FineTuning_Outputs"

trainer = SFTTrainer(
  model = model,
  tokenizer = tokenizer,
  train_dataset = dataset,
  dataset_text_field = "text",
  max_seq_length = max_seq_length,
  dataset_num_proc = 2,
  packing = False,
  args = TrainingArguments(
      # --- Parâmetros de Desempenho ---
      per_device_train_batch_size = 2,
      gradient_accumulation_steps = 4,
      warmup_steps = 5,
      # max_steps = 60, # Apenas para testes.
      num_train_epochs = 2, # Usando todo o dataset duas vezes.
      learning_rate = 2e-4, # Força a prestar muito mais atenção aos detalhes de cada exemplo.
      fp16 = not is_bfloat16_supported(),
      bf16 = is_bfloat16_supported(),
      optim = "adamw_8bit",
      weight_decay = 0.01,
      lr_scheduler_type = "linear",
      seed = 3407,

      # --- Parâmetros de LOGGING e CHECKPOINT ---
      output_dir = output_dir,          # Diretório para salvar tudo
      logging_steps = 1,                 # Mostra a 'loss' a cada passo
      report_to = "none",              # Desabilita o login do wandb
      save_strategy = "steps",         # Estratégia para salvar: a cada X passos
      save_steps = 100,                 # Salva um checkpoint a cada 100 passos
      save_total_limit = 2,            # Mantém apenas os 2 últimos checkpoints para não encher seu Drive
  ),
)

# Para iniciar o treinamento, use:
# trainer.train() # <--- Use esta linha se for a PRIMEIRA vez que está treinando
# trainer.train(resume_from_checkpoint = True) # <--- Use esta se precisar CONTINUAR um treino interrompido

In [ ]:
# Fazendo o treinamento
# E para iniciar o treinamento, use:
# trainer.train() # <--- Use esta linha se for a PRIMEIRA vez que está treinando
# trainer.train(resume_from_checkpoint = True) # <--- Use esta se precisar CONTINUAR um treino interrompido

# trainer_stats = trainer.train()
trainer_stats = trainer.train(resume_from_checkpoint = True)

# Testando o Modelo Fine-Tuned

In [ ]:
# Montar o Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Instalar as bibliotecas necessárias.
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps xformers "trl<0.9.0" peft accelerate bitsandbytes
!pip install transformers datasets

In [ ]:
# Carregando o Modelo Fine-Tuned (Método de 2 Etapas)

import torch
from unsloth import FastLanguageModel

# --- Configuração ---
max_seq_length = 2048
dtype = None
load_in_4bit = True

# --- CAMINHO PARA O ADAPTADOR TREINADO ---
# Este é o caminho para os pesos do fine-tuning
caminho_do_adaptador_lora = r"/content/drive/MyDrive/Biblioteca/Acadêmico/Pós Graduação/Pós Tech - FIAP/Aulas/Fase_3/Tech_Challenge/Tech_Challenge--5IADT--Fase_03/FineTuning_Outputs/checkpoint-12500"

# --- NOME DO MODELO BASE ORIGINAL ---
# Este é o modelo que foi usado para o treinamento inicial
nome_do_modelo_base = "unsloth/llama-3-8b-bnb-4bit"


# ETAPA 1: Carregue o modelo BASE original a partir da internet.
# Isso garante que a Unsloth tenha o 'config.json' correto e saiba que está lidando com um Llama 3.
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = nome_do_modelo_base,
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

# ETAPA 2: Aplicando o adaptador LoRA treinado (pesos) por cima do modelo base.
model.load_adapter(caminho_do_adaptador_lora)


# Prepara o modelo final para inferência (mais rápido)
FastLanguageModel.for_inference(model)
print("Modelo carregado com sucesso!")

In [ ]:
# Loop de Teste Interativo (VERSÃO CORRIGIDA PARA LLAMA-3)

from transformers import TextStreamer
import torch

# Define o template do prompt para INFERÊNCIA.
# Note que ele termina exatamente onde a resposta do assistente deve começar.
LLAMA3_INFERENCE_PROMPT = """<|begin_of_text|><|start_header_id|>user<|end_header_id|>

Com base no título do produto, gere a sua descrição.
Título: {}<|eot_id|><|start_header_id|>assistant<|end_header_id|>
"""

# Loop para testar o modelo várias vezes
while True:
    # Pede um título de produto para o usuário
    titulo_produto = input("Digite o título do produto (ou 'sair' para terminar): ")

    # Condição para sair do loop
    if titulo_produto.lower() == 'sair':
        print("Encerrando o teste.")
        break

    # Formata o prompt para o modelo usando o novo template Llama-3
    prompt = LLAMA3_INFERENCE_PROMPT.format(titulo_produto)

    # Tokeniza o prompt e o envia para a GPU
    inputs = tokenizer([prompt], return_tensors = "pt").to("cuda")

    # Usa o TextStreamer para ver a resposta sendo gerada em tempo real
    text_streamer = TextStreamer(tokenizer, skip_prompt=True) # skip_prompt=True é útil aqui
    _ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 256,
                       eos_token_id=tokenizer.eos_token_id,
                       repetition_penalty=1.15) # Adicionado para parar corretamente

    # Adiciona uma linha de separação para o próximo teste
    print("\\n" + "="*50 + "\\n")

# Para Hugging Face Hub

In [ ]:
# Montar o Google Drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Instalar as bibliotecas necessárias.
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"

In [ ]:
# SALVANDO O MODELO FINAL


import torch
from unsloth import FastLanguageModel

# --- Caminho onde o checkpoint final foi salvo ---
caminho_do_checkpoint_final = r"/content/drive/MyDrive/Biblioteca/Acadêmico/Pós Graduação/Pós Tech - FIAP/Aulas/Fase_3/Tech_Challenge/Tech_Challenge--5IADT--Fase_03/FineTuning_Outputs/checkpoint-12500"

# --- Onde salvar o modelo pronto para upload ---
caminho_para_salvar_modelo = r"/content/drive/MyDrive/Biblioteca/Acadêmico/Pós Graduação/Pós Tech - FIAP/Aulas/Fase_3/Tech_Challenge/Tech_Challenge--5IADT--Fase_03/modelo_final_Llama3_8b_TCF3"

# Carrega o modelo a partir do checkpoint final
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = caminho_do_checkpoint_final,
)

# Salva o modelo e o tokenizador em um novo diretório.
# Este comando junta o adaptador LoRA com os arquivos de configuração necessários.
model.save_pretrained(caminho_para_salvar_modelo)
tokenizer.save_pretrained(caminho_para_salvar_modelo)

print(f"Modelo final salvo com sucesso em: {caminho_para_salvar_modelo}")

In [ ]:
# UPLOAD PARA O HUGGING FACE HUB

from huggingface_hub import login
from transformers import AutoTokenizer, AutoModelForCausalLM

# token de acesso do Hugging
login("")

# --- Onde o modelo final foi salvo ---
caminho_do_modelo_salvo = r"/content/drive/MyDrive/Biblioteca/Acadêmico/Pós Graduação/Pós Tech - FIAP/Aulas/Fase_3/Tech_Challenge/Tech_Challenge--5IADT--Fase_03/modelo_final_Llama3_8b_TCF3"

# --- Nome no repositório no Hugging Face ---
nome_do_repo_hf = "robsonnicacio/llama-3-8b-amazon-descriptions-tcf3"

# Carrega o tokenizador e o modelo localmente
tokenizer = AutoTokenizer.from_pretrained(caminho_do_modelo_salvo)
model = AutoModelForCausalLM.from_pretrained(caminho_do_modelo_salvo)

# Faz o upload
tokenizer.push_to_hub(nome_do_repo_hf, use_temp_dir=True)
model.push_to_hub(nome_do_repo_hf, use_temp_dir=True)

print(f"Upload concluído! Seu modelo está disponível em: https://huggingface.co/{nome_do_repo_hf}")

# Usando o Modelo Treinado via Hugging Face Hub

In [ ]:
# Instalando as bibliotecas necessárias.
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"

In [ ]:

# Carregando o Modelo Fine-Tuned DIRETAMENTE DO HUGGING FACE
# OBS: falta a parte para inferência aqui!

import torch
from unsloth import FastLanguageModel

# --- NOME DO SEU MODELO NO HUGGING FACE ---
nome_do_seu_modelo_no_hf = "robsonnicacio/llama-3-8b-amazon-descriptions-tcf3"

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = nome_do_seu_modelo_no_hf,
    max_seq_length = 2048,
    dtype = None,
    load_in_4bit = True,
)

# Prepara o modelo final para inferência
FastLanguageModel.for_inference(model)
print("Modelo fine-tuned carregado com sucesso do Hugging Face Hub!")

In [ ]:
# Loop de Teste Interativo (Testando o que está no Hugging Face)

from transformers import TextStreamer


# Define o template do prompt para INFERÊNCIA.
# Note que ele termina exatamente onde a resposta do assistente deve começar.
LLAMA3_INFERENCE_PROMPT = """<|begin_of_text|><|start_header_id|>user<|end_header_id|>

Com base no título do produto, gere a sua descrição.
Título: {}<|eot_id|><|start_header_id|>assistant<|end_header_id|>
"""

# Loop para testar o modelo várias vezes
while True:
    # Pede um título de produto para o usuário
    titulo_produto = input("Digite o título do produto (ou 'sair' para terminar): ")

    # Condição para sair do loop
    if titulo_produto.lower() == 'sair':
        print("Encerrando o teste.")
        break

    # Formata o prompt para o modelo usando o novo template Llama-3
    prompt = LLAMA3_INFERENCE_PROMPT.format(titulo_produto)

    # Tokeniza o prompt e o envia para a GPU
    inputs = tokenizer([prompt], return_tensors = "pt").to("cuda")

    # Usa o TextStreamer para ver a resposta sendo gerada em tempo real
    text_streamer = TextStreamer(tokenizer, skip_prompt=True) # skip_prompt=True é útil aqui
    _ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 256,
                       eos_token_id=tokenizer.eos_token_id,
                       repetition_penalty = 1.15) # Adicionado para parar corretamente

    # Adiciona uma linha de separação para o próximo teste
    print("\\n" + "="*50 + "\\n")

# Teste em uma Única Céula

In [ ]:
# Etapa 1: Instalar as bibliotecas necessárias
# Descomente a linha abaixo se estiver em um novo ambiente Colab
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"

import torch
from unsloth import FastLanguageModel
from transformers import TextStreamer, StoppingCriteria, StoppingCriteriaList

# --- Etapa 2: Carregar o Modelo do Hugging Face Hub ---

# O nome do seu repositório no Hugging Face
nome_do_seu_modelo_no_hf = "robsonnicacio/llama-3-8b-amazon-descriptions-tcf3"

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = nome_do_seu_modelo_no_hf,
    max_seq_length = 2048,
    dtype = None,
    load_in_4bit = True,
)

# Prepara o modelo para uma inferência mais rápida
FastLanguageModel.for_inference(model)
print("Modelo fine-tuned carregado com sucesso do Hugging Face Hub!")


# --- Etapa 3: Configurar a Inferência de Forma Robusta ---

# Template de prompt para a inferência
LLAMA3_INFERENCE_PROMPT = """<|begin_of_text|><|start_header_id|>user<|end_header_id|>

Com base no título do produto, gere a sua descrição.
Título: {}<|eot_id|><|start_header_id|>assistant<|end_header_id|>
"""

# Definição dos Critérios de Parada (Stopping Criteria) para uma saída limpa
stop_tokens = ["<|eot_id|>", "<|end_of_text|>"]
stop_token_ids = [tokenizer.convert_tokens_to_ids(token) for token in stop_tokens]

class StopOnTokens(StoppingCriteria):
    def __call__(self, input_ids: torch.LongTensor, scores: torch.FloatTensor, **kwargs) -> bool:
        for stop_id in stop_token_ids:
            if input_ids[0][-1] == stop_id:
                return True
        return False

stopping_criteria = StoppingCriteriaList([StopOnTokens()])


# --- Etapa 4: Loop de Teste Interativo ---
while True:
    titulo_produto = input("Digite o título do produto (ou 'sair' para terminar): ")

    if titulo_produto.lower() == 'sair':
        print("Encerrando o teste.")
        break

    prompt = LLAMA3_INFERENCE_PROMPT.format(titulo_produto)
    inputs = tokenizer([prompt], return_tensors = "pt").to("cuda")

    # Usamos o TextStreamer para ver a resposta sendo gerada em tempo real
    # skip_special_tokens=True limpa a saída de tokens como <|eot_id|>
    text_streamer = TextStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)

    _ = model.generate(
        **inputs,
        streamer = text_streamer,
        max_new_tokens = 256,
        stopping_criteria = stopping_criteria, # Usa o critério de parada robusto
        repetition_penalty = 1.15
    )

    print("\n" + "="*50 + "\n")
